In [1]:
import numpy as np
import bundle
import time
import ray
from tqdm import tqdm

## update cluster environments and files

In [2]:
import os
os.environ["KUBECONFIG"] = "./config/magics_cluster.yaml" #kubernetes
! ray up config/cluster.yaml -y # update ray cluster + service.yaml

Cluster: toy-cluster

Setting `max_workers` for node type `head_node` to the global `max_workers` value of 4. To avoid spawning worker nodes of type `head_node`, explicitly set `max_workers: 0` for `head_node`.
Note that `max_workers: 0` was the default value prior to Ray 1.3.0. Your current version is Ray 1.6.0.
See the docs for more information:
https://docs.ray.io/en/master/cluster/config.html#cluster-configuration-node-max-workers
https://docs.ray.io/en/master/cluster/config.html#full-configuration
Loaded cached provider configuration
If you experience issues with the cloud provider, try re-running the command with --no-config-cache.
Updating cluster configuration and running full setup.
Cluster Ray runtime will be restarted. Confirm [y/N]: y [automatic, due to --yes]

<1/1> Setting up head node
  Prepared bootstrap config
  New status: waiting-for-ssh
  [1/7] Waiting for SSH to become available
    Running `uptime` as a test.
2021-09-02 00:10:35,192	INFO command_runner.py:172 -- N

In [8]:
ray.init(address="ray://192.168.100.108:32703")

ClientContext(dashboard_url='192.168.78.51:8265', python_version='3.8.5', ray_version='1.6.0', ray_commit='b2b2901dd3144bf6ee10b01912353dd826eaa510', protocol_version='2021-05-20', _num_clients=1)

In [11]:
@ray.remote
def ray_bundles(idx):
    bundle_list, Z_list = ray.get(ray_env).create_bundles(idx,idx+1)
    return  bundle_list,Z_list
t1 = time.time()
env = bundle.Evaluator()
ray_env = ray.put(env)
return_refs = [ray_bundles.remote(idx) for idx in range(260, 265)]
return_items = ray.get(return_refs)
print(f"Total time = {time.time() - t1}")

init a new Data Manager here
(pid=4391) 260
(autoscaler +3m58s) Tip: use `ray status` to view detailed autoscaling status. To disable autoscaler event messages, you can set AUTOSCALER_EVENTS=0.
(autoscaler +3m58s) Adding 2 nodes of type head_node.
(pid=2132, ip=192.168.72.14) 261
(pid=2139, ip=192.168.72.13) 262
(autoscaler +4m4s) Adding 2 nodes of type head_node.
(pid=4391) 263
(autoscaler +4m10s) Adding 1 nodes of type head_node.
(pid=2139, ip=192.168.72.13) 264
Total time = 58.22618770599365


In [ ]:
B_items = [return_items[i][0][0] for i in range(len(return_items))]
Z_items = [return_items[i][1][0] for i in range(len(return_items))]
import pickle
B_file = "./Data/B_list.pkl"
Z_file = "./Data/Z_list.pkl"
with open(B_file, 'wb') as f:
    pickle.dump(B_items, f)
with open(Z_file, 'wb') as f:
    pickle.dump(Z_items, f)

## submit job to cluster

In [ ]:
import os
os.environ["KUBECONFIG"] = "./config/magics_cluster.yaml"
!ray rsync_up config/cluster.yaml "./Data" "~/."
!ray submit config/cluster.yaml train.py 
!ray rsync_down config/cluster.yaml '~/Data/Z_list.pkl' './Data/.'

Setting `max_workers` for node type `head_node` to the global `max_workers` value of 4. To avoid spawning worker nodes of type `head_node`, explicitly set `max_workers: 0` for `head_node`.
Note that `max_workers: 0` was the default value prior to Ray 1.3.0. Your current version is Ray 1.6.0.
See the docs for more information:
https://docs.ray.io/en/master/cluster/config.html#cluster-configuration-node-max-workers
https://docs.ray.io/en/master/cluster/config.html#full-configuration
Loaded cached provider configuration
If you experience issues with the cloud provider, try re-running the command with --no-config-cache.
Setting `max_workers` for node type `head_node` to the global `max_workers` value of 4. To avoid spawning worker nodes of type `head_node`, explicitly set `max_workers: 0` for `head_node`.
Note that `max_workers: 0` was the default value prior to Ray 1.3.0. Your current version is Ray 1.6.0.
See the docs for more information:
https://docs.ray.io/en/master/cluster/config.htm

In [2]:
import os
os.environ["KUBECONFIG"] = "./config/magics_cluster.yaml" #kubernetes
! ray down config/cluster.yaml -y

Setting `max_workers` for node type `head_node` to the global `max_workers` value of 4. To avoid spawning worker nodes of type `head_node`, explicitly set `max_workers: 0` for `head_node`.
Note that `max_workers: 0` was the default value prior to Ray 1.3.0. Your current version is Ray 1.6.0.
See the docs for more information:
https://docs.ray.io/en/master/cluster/config.html#cluster-configuration-node-max-workers
https://docs.ray.io/en/master/cluster/config.html#full-configuration
Loaded cached provider configuration
If you experience issues with the cloud provider, try re-running the command with --no-config-cache.
Destroying cluster. Confirm [y/N]: y [automatic, due to --yes]
Setting `max_workers` for node type `head_node` to the global `max_workers` value of 4. To avoid spawning worker nodes of type `head_node`, explicitly set `max_workers: 0` for `head_node`.
Note that `max_workers: 0` was the default value prior to Ray 1.3.0. Your current version is Ray 1.6.0.
See the docs for mor

## TIPS for control ray cluster
sync files:   ray rsync_up cluster.yaml 'bundle.py' '/home/ray/dev_ws/bundle.py'

update denpendency:  ray up cluster.yaml

connect through ssh:  ray attach cluster.yaml

execute command on head: ray exec cluster.yaml ""